### Horovod fundamentals

https://github.com/horovod/horovod#documentation

How to convert a single node training script to multi-node.

1. Run hvd.init() to initialize Horovod
2. Pin each GPU to single process (using local rank)
3. Scale learning rate by number of workers (LR is given as part of optimizer)
4. Wrap optimizer in hvd.DistributedOptimizer
5. Broadcast initial variables from rank 0 to other ranks.
6. Save checkpoint only on rank 0 to avoid corruption.

# Structure of a TF training script using keras

# Initialize horovod
hvd.init()

# Pin process to dedicated GPU
config = tf.ConfigProto()
config.gpu_options.visible_device_list = str(hvd.local_rank())
keras.backend.set_session(config=config)

# Prepare dataset
x_train, x_test, y_train, y_test

# Define the model
model = Sequential()
model.add(...)

# Define optimizer and scale learning rate
opt = keras.optimizers.Adadelta(lr * hvd.size())

# Add distributed optimizer
opt = hvd.DistributedOptimizer(opt, backward_passes_per_step=1)

# Tie together the loss function, optimizer, metric, and model
model.compile(loss_fn, opt, metrics=['accuracy'])

# Custom callbacks during training for broadcasting initial params, checkpointing
callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),
]
# Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
if hvd.rank() == 0:
    callbacks.append(keras.callbacks.ModelCheckpoint('./checkpoint-{epoch}.h5'))
    
# Start training
model.fit(x_train, y_train,
          batch_size=batch_size,
          callbacks=callbacks,
          epochs=epochs,
          verbose=1 if hvd.rank() == 0 else 0,
          validation_data=(x_test, y_test))

Efficient Net paper
https://arxiv.org/pdf/1905.11946.pdf


Github link -https://github.com/NVIDIA/DeepLearningExamples/tree/master/TensorFlow2/Classification/ConvNets/efficientnet


Special Features  -
- Mixed precision arithmetic
- XLA
- cosine decay LR
- multi GPU Horovod with NCCL backend
- TF 32 in A100

How to prepare dataset
Use https://github.com/kmonachopoulos/ImageNet-to-TFrecord/blob/master/build_imagenet_data.py to generate TF records
python convert.py --train_directory=/balarjun/pytorch/train --validation_directory=/balarjun/pytorch/val
Uploaded TF records to s3://smddp-570106654206-us-west-2/dataset/efficient/

#### Setting up DLC and running Horovod TF training
1. Download DLC 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.6.0-gpu-py38-cu112-ubuntu20.04 and extract. Follow our guide https://quip-amazon.com/RGLAANbEf82L#Jfb9AA6x7dX for this.
2. Inside fsx, setup code https://github.com/NVIDIA/DeepLearningExamples and download TF records (this will be set as data dir)    
3. srun -N 1 nvidia-modprobe -u -c=0
3. Login to DLC: srun -N 1 --pty enroot start --rw --root -m /fsx:/fsx pyxis_dlc-tf
4. cd /fsx/DeepLearningExamples/TensorFlow2/Classification/ConvNets/efficientnet/
5.
pip install nvidia-pyindex
pip install nvidia-dllogger
pip install tensorflow-addons
pip install tensorflow-datasets
pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110
pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-tf-plugin-cuda110
6. python main.py --mode=train --arch=efficientnet-b0 --model_dir=/fsx/ --data_dir=/fsx/efficient-net-data/
check that above code can start running
7. Running inside container, multi process
mpirun --hostfile hostfile -N 8 --tag-output --oversubscribe --allow-run-as-root --mca btl_vader_single_copy_mechanism none--mca btl_tcp_if_exclude lo,docker0 -x PATH=/opt/amazon/openmpi/bin:$PATH -x LD_LIBRARY_PATH=/opt/amazon/openmpi/lib:$LD_LIBRARY_PATH  -x NCCL_DEBUG=INFO -x RDMAV_FORK_SAFE=1 python main.py --mode=train --arch=efficientnet-b0 --model_dir=/fsx/ --data_dir=/fsx/efficient-net-data/

#### Adaptation to SM DDP
SMDDP adaptation code is at this branch - https://github.com/Arjunbala/DeepLearningExamples/tree/smddp-adapt
It has 2 commits. (one main conversion and one some bug fixes)
Next, install herring inside TF container. Had to modify build script to remove SMDATAPARALLEL_PT option.

Running in distributed mode on EC2 -
Needed to do a bunch of hacks to get things working.
Remove -c /opt/conda from exec_inside_alloc.sh
Got "Unable to activate conda environment error" -- removed the exit 1 call and allowed execution to proceed made things work

hrun -N 2 --container dlc-tf python /fsx/adaptation/DeepLearningExamples/TensorFlow2/Classification/ConvNets/efficientnet/main.py --mode=train --arch=efficientnet-b0 --model_dir=/fsx/ --data_dir=/fsx/efficient-net-data/